In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matrix_loader as loader

# Final Project Journal
*Luna Yee (with Nathaniel Sauerberg)*

## Project Overview

Our goal is to expand upon the work of Griffiths *et al.* by comparing other web search ranking algorithms to Google's PageRank in order to test whether PageRank is the best such model, or if another algorithm such as HITS could more closely resemble how we rank items.

## Project Log

### Project Log: Oct 21

Nathaniel and I reviewed the scholarly articles behind our project. Primarily, we looked to Griffiths *et al.*'s <a href="">Google and the Mind</a> to better understand the methods that Griffiths *et al.* used to create and process the semantic network of the association data from Nelson, McEvoy and Schreiber. Our upcoming goal is to develop functions to (1) read the data into memory, (2) sort this data into an association matrix, and (3) make this data accessible in a format that is useful for our search algorithms.

As might be clear from the text, step (3) is the least well defined. We are currently looking into the scholarly publication of <a href="">Google's PageRank and Beyond</a>, which describes how PageRank and other such search engine ranking algorithms function. This will help us to define how to shape the data from the matrix into a useful format.

### Project Log: Oct 27

I have begun to assemble the basis for our semantic network program. First and foremost, we now have all of the data of cue-target pairs from Nelson, McEvoy and Schreiber assembled into on text file. Now, this has its share of complications: first of all, the sheer size of the data is large enough that certain platforms (i.e. GitHub) will not display it naturally; the data consists of over 72000 lines of text. Second, the format of the data is very specific and can be arcane. It is described in full in [Appendix A of Nelson *et al.*'s website](http://w3.usf.edu/FreeAssociation/AppendixA/), but we will probably only be concerned with the first few fields:

```
CUE, TARGET, NORMED?, #G, #P, FSG, ...
```

The `CUE` and `TARGET` represent the word given and the word produced, respectively, which will describe the links. `NORMED?` tells whether the `TARGET` is later studied as a `CUE`, which will be important because links that have `NO` for this field will be dead ends in the random surfer paradigm. `FSG` is the forward strength of the link, which is a stochastic measure of the proportion of times this target was produced from the set of all responses for the cue given. That is, it is the ratio of `#P`, the number of participants producing this target, over `#G`, the number of participants tested on this cue.

I have also started to code the python module we will use for loading. The plan is to store most of the data in a dictionary as it loads in, then convert this dictionary into a matrix in a separate method call (since the time cost will likely be heavy). The problems to address tomorrow: what format do we want for the output, or do we want to store natively (in an instance)? how can we deal with the sheer volume of data elegantly? once the matrix is made, what do we need to have it do? the Griffiths *et al.* paper mentioned directed graphs: how would we represent these in Python?

### Project Log: Oct 28

Nathaniel and I have worked out the flaws in the module I started, and he and I have expanded upon it. After discussion over Griffiths *et al.*, we have decided that we do want to include the forward strengths&mdash;though Nathaniel had the insight that we would want to calculate this by dividing `#P / #G` to get a more accurate, unrounded value&mdash;but that we do not know if we should include unnormed targets in the unweighted implementation. To clarify: Griffiths *et al.* did two passes: the first was the unweighted PageRank, which is the classic implementation that takes the eigenvectors. The second was a weighted implementation, for which they specified that they 'dealt with' the hanging link problem in some manner. Since we do not know if they pruned these hanging links from the unweighted data, we are planning now to produce four matrices: (1) unweighted without unnormed; (2) unweighted with unnormed; (3) weighted without unnormed; and (4) weighted with unnormed. This should not be too difficult from a programmatic standpoint, but we do need to make indexing lists for these matrices, which will look slightly different for the ones with the unnormed targets.

We also discussed some options: first, Nathaniel suggested a functional approach to the module, where I was implementing an object-oriented approach. This makes a lot of sense from a utility standpoint, so we will work to transmute the format. Second, Nathaniel brought up the `pickle` module for Python. This seems an ideal way to deal with the issue of volume of data, seeing as how `pickle` compresses objects into byte streams, and then unpackages them from that format. This would make passing around and storing the matrix and any other association data far more manageable. However, I need to do [research into this module](https://docs.python.org/3/library/pickle.html), since I have not used it myself, rather only seen it used.

### Project Log: Oct 30

I have converted the object-oriented `load()` method to a functional approach. I have also tested that it can successfully operate on the data&mdash;after a few iterations of bug fixes, it will load the text file without complaint. One bug fix was a hanging description field in the data that needed to be removed. The others were largely syntactical. My next step is to ensure that the function is indeed outputting the desired format, for which I will need to create an abbreviated test file, since the full data is much too long. Notably, however, it seems that the run time on loading is not unmanageably long, probably since it is almost entirely plain-text processing, which is something to be thankful for. However, we will still look into `pickle`.

### Project Log: Nov 2

The testing of the `load()` method is included below. It is now (after a couple of fixes) working as intended. `data/cue-target-pairs_A.txt` now contains an abbreviated set of data (cues from the letter A) to use for testing. Using this code to test processing times for the full file, loading takes less than five seconds, so `load()` should not be very costly to run (which also helps affirm that it is working as intended; a long run time might indicate some faults in the algorithm). Creating the matrix may be significantly more costly, however; testing the matrix creation methods should be our next step.

In [20]:
test_file = 'data/cue-target-pairs_A.txt'
assocs, normed_list, unnormed_list = loader.load(test_file)
# Print data with formatting; also assert-check the two lists for completion
for key in assocs:
    assert key in normed_list # Ensure all cues are in normed_list
    value = assocs[key]
    print('Targets for the cue "{0}":'.format(key))
    for tup in value:
        target, fsg, normed = tup
        print(' {0:>20s}: {1:>8s} with forward strength {2:6f}'.format(
              '"' + target + '"', 'NORMED' if normed else 'UNNORMED', fsg))
        if not normed:
            assert target in unnormed_list # Ensure all unnormed targets are in unnormed_list

Loading cue-target pairs from file: 'data/cue-target-pairs_A.txt'
Load complete: 3832 cue-target lines parsed, yielding 269 total cues
Targets for the cue "AWAY":
                "FAR":   NORMED with forward strength 0.229730
               "GONE":   NORMED with forward strength 0.121622
               "HOME":   NORMED with forward strength 0.087838
               "HERE":   NORMED with forward strength 0.074324
               "NEAR":   NORMED with forward strength 0.074324
              "CLOSE":   NORMED with forward strength 0.040541
               "FROM": UNNORMED with forward strength 0.033784
               "STAY":   NORMED with forward strength 0.033784
                 "GO":   NORMED with forward strength 0.027027
              "LEAVE":   NORMED with forward strength 0.027027
           "VACATION":   NORMED with forward strength 0.027027
               "COME":   NORMED with forward strength 0.020270
             "SCHOOL":   NORMED with forward strength 0.020270
           "TOGETH

               "LOOK":   NORMED with forward strength 0.010989
Targets for the cue "ARITHMETIC":
               "MATH":   NORMED with forward strength 0.756944
           "CALCULUS":   NORMED with forward strength 0.034722
            "ALGEBRA":   NORMED with forward strength 0.027778
             "NUMBER":   NORMED with forward strength 0.027778
                "ADD":   NORMED with forward strength 0.020833
              "LOGIC":   NORMED with forward strength 0.013889
Targets for the cue "ARGUMENT":
              "FIGHT":   NORMED with forward strength 0.513514
               "YELL":   NORMED with forward strength 0.081081
           "DISAGREE":   NORMED with forward strength 0.033784
       "DISAGREEMENT":   NORMED with forward strength 0.033784
              "ANGER":   NORMED with forward strength 0.020270
                "MAD":   NORMED with forward strength 0.020270
            "QUARREL":   NORMED with forward strength 0.020270
             "DEBATE":   NORMED with forward strengt

           "EMPLOYEE":   NORMED with forward strength 0.014388
                "JOB":   NORMED with forward strength 0.014388
               "KNOW":   NORMED with forward strength 0.014388
                "LAW":   NORMED with forward strength 0.014388
              "MATCH":   NORMED with forward strength 0.014388
             "MINGLE": UNNORMED with forward strength 0.014388
             "RELATE": UNNORMED with forward strength 0.014388
           "TOGETHER":   NORMED with forward strength 0.014388
               "WITH":   NORMED with forward strength 0.014388
Targets for the cue "ASSISTANCE":
               "HELP":   NORMED with forward strength 0.695946
                "AID":   NORMED with forward strength 0.054054
             "HELPER":   NORMED with forward strength 0.054054
          "SECRETARY":   NORMED with forward strength 0.020270
          "FINANCIAL":   NORMED with forward strength 0.013514
           "HANDICAP":   NORMED with forward strength 0.013514
Targets for the cue "